# Model Denemeleri

### Önce HuggingFace'ten, hali hazırda tamamen türkçe veriler üzerinde eğitilmiş BerTurk modeli denenecek

In [1]:
from transformers import AutoTokenizer, BertForPreTraining
import torch

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForPreTraining.from_pretrained("dbmdz/bert-base-turkish-cased")

In [2]:
sd_hf = model.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [3]:
torch.equal(sd_hf["bert.embeddings.word_embeddings.weight"], sd_hf["cls.predictions.decoder.weight"]) # tying kontrolü

True

In [4]:
sd_hf["bert.embeddings.word_embeddings.weight"].data_ptr() == sd_hf["cls.predictions.decoder.weight"].data_ptr()  # tying kontrolü

True

In [5]:
torch.equal(sd_hf["cls.predictions.decoder.bias"], sd_hf["cls.predictions.bias"]) # tying kontrolü

True

In [6]:
sd_hf["cls.predictions.decoder.bias"].data_ptr() == sd_hf["cls.predictions.bias"].data_ptr()  # tying kontrolü

True

In [7]:
inputs = tokenizer("Selamlar efendim", return_tensors="pt")
outputs = model(**inputs)

In [8]:
outputs.prediction_logits.shape, outputs.seq_relationship_logits.shape

(torch.Size([1, 4, 32000]), torch.Size([1, 2]))

In [9]:
from transformers import pipeline

fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [10]:
from model import BertForPreTraining, BertConfig

%load_ext autoreload
%autoreload 2

In [11]:
default_config = BertConfig()
custom_model = BertForPreTraining(default_config)

In [12]:
custom_sd = custom_model.state_dict()

assert len(custom_sd) == len(sd_hf)

for custom, hf in zip(custom_sd.items(), sd_hf.items()):
    k, v = custom
    k_hf, v_hf = hf
    assert k == k_hf, f"{k} : {k_hf}"
    assert v.shape == v_hf.shape, f"{k} : {v.shape} != {v_hf.shape}"
    print(k, v.shape, "--------", k_hf, v_hf.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768]) -------- bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768]) -------- bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768]) -------- bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768]) -------- bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768]) -------- bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768]) -------- bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768]) -------- bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768]) -

In [13]:
model_from_hf = BertForPreTraining.from_pretrained()
mm = model_from_hf.state_dict()

loading weights from pretrained bert: dbmdz/bert-base-turkish-cased


In [44]:
from transformers import pipeline

torch.random.manual_seed(42)
fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?") 

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [64]:
fill_masker("ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı") 

[{'score': 0.9745508432388306,
  'token': 22777,
  'token_str': 'Muhammet',
  'sequence': 'ışınlanma teknolojisini Muhammet Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.016110757365822792,
  'token': 7397,
  'token_str': 'Muhammed',
  'sequence': 'ışınlanma teknolojisini Muhammed Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.0018351072212681174,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'ışınlanma teknolojisini Mehmet Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.00045096001122146845,
  'token': 6001,
  'token_str': 'Hasan',
  'sequence': 'ışınlanma teknolojisini Hasan Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.0004167473816778511,
  'token': 6391,
  'token_str': 'Cem',
  'sequence': 'ışınlanma teknolojisini Cem Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'}]

In [50]:
# WITHOUT PAD TOKEN
from model import FillMaskPipeline, IsNextPipeline
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "[MASK] "])

encoding_input_id_tensor device:  cpu
encoding_input_id_tensor dtype:  torch.int64
model device:  cpu
Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'hanım', 'Salih', 'hoca', 'doktor'], 'score': ['0.0415', '0.0369', '0.0276', '0.0280', '0.0300']}
Text: [MASK]  ----> Top 5 Predictions: {'token_str': ['.', ')', '!', '"', ']'], 'score': ['0.6554', '0.0445', '0.0148', '0.0511', '0.0232']}


In [53]:
# WITH PAD TOKEN
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı"])

encoding_input_id_tensor device:  cpu
encoding_input_id_tensor dtype:  torch.int64
model device:  cpu
Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'hanım', 'Salih', 'hoca', 'doktor'], 'score': ['0.0415', '0.0369', '0.0276', '0.0280', '0.0300']}
Text: ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı ----> Top 5 Predictions: {'token_str': ['Muhammet', 'Muhammed', 'Mehmet', 'Cem', 'Hasan'], 'score': ['0.9561', '0.0241', '0.0035', '0.0007', '0.0011']}


In [54]:
# WITH PAD TOKEN
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı"])

encoding_input_id_tensor device:  cpu
encoding_input_id_tensor dtype:  torch.int64
model device:  cpu
Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'hanım', 'Salih', 'hoca', 'doktor'], 'score': ['0.0415', '0.0369', '0.0276', '0.0280', '0.0300']}
Text: ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı ----> Top 5 Predictions: {'token_str': ['Muhammet', 'Muhammed', 'Mehmet', 'Cem', 'Hasan'], 'score': ['0.9561', '0.0241', '0.0035', '0.0007', '0.0011']}


In [55]:
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?"])

encoding_input_id_tensor device:  cpu
encoding_input_id_tensor dtype:  torch.int64
model device:  cpu
Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'hanım', 'Salih', 'hoca', 'doktor'], 'score': ['0.0415', '0.0369', '0.0276', '0.0280', '0.0300']}


In [56]:
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="multinomial")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisi [MASK] gitmektedir."])

encoding_input_id_tensor device:  cpu
encoding_input_id_tensor dtype:  torch.int64
model device:  cpu
Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['bey', 'hanım', 'Mehmet', 'Selim', 'Salih'], 'score': ['0.0267', '0.0369', '0.0415', '0.0030', '0.0276']}
Text: ışınlanma teknolojisi [MASK] gitmektedir. ----> Top 5 Predictions: {'token_str': ['yavaş', '##yle', 'yolunda', 'nereye', 'birlikte'], 'score': ['0.0113', '0.0707', '0.0077', '0.0034', '0.0047']}


In [57]:
# random initialized, so garbage output expected
fill_mask_custom = FillMaskPipeline(model=BertForPreTraining(BertConfig()), tokenizer=tokenizer, strategy="greedy")    
fill_mask_custom(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisi [MASK] gitmektedir."]) 

encoding_input_id_tensor device:  cpu
encoding_input_id_tensor dtype:  torch.int64
model device:  cpu
Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['##düğüm', 'Tahmin', 'programlar', 'Konuyla', 'kesilen'], 'score': ['0.0002', '0.0002', '0.0002', '0.0005', '0.0002']}
Text: ışınlanma teknolojisi [MASK] gitmektedir. ----> Top 5 Predictions: {'token_str': ['metal', 'ezilen', 'Konuyla', '##ym', '##altına'], 'score': ['0.0003', '0.0003', '0.0003', '0.0003', '0.0003']}


In [65]:
nsp_pipeline = IsNextPipeline(model=model_from_hf, tokenizer=tokenizer)

In [66]:
nsp_pipeline([["Çocuk sahibi çift sayısında inanılmaz bir artış var.", "Uzaya ilk Bekiroğ Reis çıkmıştır"],
              ["Bu nsp olayı bir garip oldu. Sanki kablolar ters bağlandı.", "Bir şekilde kabloları ayarlamak gerekli"],
              ["çoğunuz yaş itibariyle tanımaz ama istanbul'un en iyi belediye başkanı justinianus'tur.", "Samsun Ayvacık belediyesi ne yapmak nereye varmak istemektedir!"] ])

Text: ['Çocuk sahibi çift sayısında inanılmaz bir artış var.', 'Uzaya ilk Bekiroğ Reis çıkmıştır'] Predictions ----> isNext: 0.072, notNext: 0.928
Text: ['Bu nsp olayı bir garip oldu. Sanki kablolar ters bağlandı.', 'Bir şekilde kabloları ayarlamak gerekli'] Predictions ----> isNext: 1.000, notNext: 0.000
Text: ["çoğunuz yaş itibariyle tanımaz ama istanbul'un en iyi belediye başkanı justinianus'tur.", 'Samsun Ayvacık belediyesi ne yapmak nereye varmak istemektedir!'] Predictions ----> isNext: 0.056, notNext: 0.944
